In [1]:
import os
from dotenv import load_dotenv
import tempfile
from chromadb import Client
from chromadb.config import Settings

from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import MultiRetrievalQAChain


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
#오픈AI API 키 설정
load_dotenv()  # 현재 경로의 .env 로드
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
if not os.environ['OPENAI_API_KEY']:
    raise ValueError('OPENAI_API_KEY not found in environment. set it in .env or env vars')

In [3]:
file_path = '../data/langchainThon/cv/경력기술서_천승우.docx'
# 파일을 바이너리 모드로 읽어서 _file처럼 흉내내기
class DummyFile:
    def __init__(self, path):
        self.name = path
        with open(path, "rb") as f:
            self._data = f.read()
    def getvalue(self):
        return self._data

_file = DummyFile(file_path)

In [4]:
# CV 불러오기 (PDF/Word)
def load_cv(_file):
    with tempfile.NamedTemporaryFile(mode='wb', delete=False) as tmp_file:
        tmp_file.write(_file.getvalue())
        tmp_file_path = tmp_file.name
        
    #PDF 파일 업로드
    if _file.name.endswith('.pdf'):
        loader = PyPDFLoader(file_path=tmp_file_path)
    
    #Word 파일 업로드
    elif _file.name.endswith('.docx'):
        loader = Docx2txtLoader(file_path=tmp_file_path)
    
    # 파일 형식이 틀릴경우 에러 메세지 출력
    else:
        raise ValueError("지원하지 않는 파일 형식입니다. PDF 또는 DOCX만 업로드해주세요.")
    
    pages = loader.load()
    return pages

# JD 불러오기 (URL)
def load_jd(_url: str):
    import warnings
    from urllib3.exceptions import InsecureRequestWarning
    warnings.simplefilter('ignore', InsecureRequestWarning)
    
    loader = WebBaseLoader(_url)
    loader.requests_kwargs = {'verify': False}  # SSL 검증 비활성화
    return loader.load()

In [5]:
def chunk_documents(docs, source_label, chunk_size=500, chunk_overlap=100):
    '''
    페이지(Document list)들을 청크로 나누고 metadata를 추가하는 함수
    - docs: Document list
    - source_label: 'cv' 또는 'jd' 등 source 표시
    - return: 청크가 나뉜 Document 리스트
    '''
    text_splitter = RecursiveCharacterTextSplitter(
        separators=['\n\n', '\n', ' '],
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        is_separator_regex=False,
    )

    split_docs = text_splitter.split_documents(docs)

    split_docs_with_meta = [
        Document(page_content=d.page_content,
                 metadata={'source': source_label, 'chunk_id': str(i)},)
        for i, d in enumerate(split_docs)
    ]

    return split_docs_with_meta

In [6]:
cv_docs = chunk_documents(load_cv(_file), 'cv')

In [7]:
jd_docs = chunk_documents(load_jd('https://www.wanted.co.kr/wd/297589'), 'jd')

In [8]:
# 클라이언트 및 임베딩 모델 초기화
client_chroma = Client(Settings(is_persistent=False))
embeddings  = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Chroma.from_documents(
    documents=cv_docs+jd_docs,
    embedding=embeddings,
    client=client_chroma,
)

In [51]:
# -----------------------------
# Retriever 설정
# -----------------------------
# CV Retriever
cv_retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'filter': {'source': 'cv'}}
)
# JD Retriever
jd_retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'filter': {'source': 'jd'}}
)
# Default Retriever
default_retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 6}
)

# -----------------------------
# System prompt
# -----------------------------
system_prompt = '''
당신은 커리어 및 채용 관련 질의응답을 돕는 AI 어시스턴트입니다.  
아래 제공된 문서들을 참고하여, 사용자의 질문에 정확하고 논리적으로 답변하세요.  

- 사용자의 **경력, 경험, 이력 관련 질문**이라면 CV(이력서)를 우선적으로 참고하세요.  
- **지원하는 회사, 직무, 채용 요건** 관련 질문이라면 JD(공고문)를 우선적으로 참고하세요.  
- 질문의 맥락상 두 문서가 모두 관련되어 있다면, **균형 있게 통합하여 답변**하세요.  
- 문서에 직접적인 정보가 없을 경우, 일반적인 HR/커리어 상식과 논리를 기반으로 보완해 설명할 수 있습니다.
- 자기소개서 작성 요청시 아래와 같은 지침을 따릅니다.

** 자기소개서 작성 지침**
- 자연스럽고 억지스럽지 않은 문장으로 꼭 필요한 부분에만 CV 내용을 활용합니다.
- CV의 내용을 적절히 활용하여 JD상의 회사 직무 기여 방안을 적절히 어필할 수 도록 합니다.
- 자기서개서 작성시 CV에 나타나는 다양한 경험들을 질문에 대한 답변에 잘 녹아들도록 활용합니다.
- 제공된 문서에만 제한을 두지 말고 일반적으로 우수하다고 생각되는 자기소개서 형태로 작성합니다.
- 자기소개서 작성시 글자수 제한은 한글을 기준으로 띄어쓰기를 포함하여 계산합니다.
- 질문지에 질문의 항목 또는 번호가 있더라도, 문장으로만 답을 합니다.

**언어 스타일 지침**
- 답변은 언제나 **전문적이고 신뢰감 있는 어투**로 작성합니다.  
- 문장은 명료하고 간결하게 유지하되, 비즈니스 상황에 맞는 적절한 어휘를 사용합니다.  
- 필요 시 가벼운 이모지를 활용하여 자연스럽게 친근함을 더할 수 있으나, 과도하게 사용하지 않습니다.  
    
문서 내용:
{context}
'''

# -----------------------------
# ChatPromptTemplate 생성
# -----------------------------
qa_prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('human', '{question}')
])


# -----------------------------
# MultiRetrievalQAChain 구성
# -----------------------------
retriever_infos = [
    {
        'name': 'cv',
        'description': 'user의 경력(CV) 및 이전 회사 관련 질문',
        'retriever': cv_retriever,
        'prompt': qa_prompt
    },
    {
        'name': 'jd',
        'description': '지원하는 회사 및 직무기술서(JD) 관련 질문',
        'retriever': jd_retriever,
        'prompt': qa_prompt
    },
]

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.4)

rag_chain = MultiRetrievalQAChain.from_retrievers(
    llm=llm,
    retriever_infos=retriever_infos,
    default_retriever=default_retriever,
)


In [49]:
query = '내가 다녔던 회사 이름과 근무 기간을 맞춰봐. 추가로 이전 회사에서 했던 주요 프로젝트 네가지 간단히 요약해줘.'
response = rag_chain.invoke({'input': query})

# LLM 답변
print('🧠 GPT-4o-mini 답변:')
print(response['result'])  # MultiRetrievalQAChain는 output_keys=['result']

🧠 GPT-4o-mini 답변:
당신이 다녔던 회사 이름은 **(주)온코크로스**이며, 근무 기간은 **2021년 1월부터 2024년 12월까지**입니다.

이전 회사에서 수행했던 주요 프로젝트 네 가지는 다음과 같습니다:

1. **항암제 신규 적응증 탐색 플랫폼 개발 프로젝트**:
   - 플랫폼 전체 기능을 Python으로 재구현하고, 전사체 데이터 및 의료 메타데이터를 관리 및 분석하는 프로젝트입니다. 공공 데이터베이스로부터 24,000개 샘플 데이터를 수집하여 대규모 분석 기반을 구축했습니다.

2. **엑체생검을 통한 암 조기 진단 모델 개발**:
   - cfDNA 및 ctDNA 데이터를 수집, 관리 및 분석하고, 암 여부 예측을 위한 딥러닝 모델을 개발하는 프로젝트입니다. 3,000개 이상의 샘플 데이터를 활용하여 CNN 모델을 훈련했습니다.

3. **데이터 전처리 및 알고리즘 고도화 프로젝트**:
   - 생명·의학 데이터를 효율적으로 처리하고 분석하기 위한 데이터 전처리 및 알고리즘을 개선하는 프로젝트입니다. 이를 통해 의사결정 지원을 위한 데이터 분석 역량을 강화했습니다.

4. **스타트업 코스닥 상장 지원 프로젝트**:
   - 스타트업의 코스닥 상장을 위한 데이터 분석 및 플랫폼 구축을 지원하는 프로젝트로, 실제 제약사와의 협업을 통해 실무 경험을 쌓았습니다.

이와 같은 프로젝트를 통해 다양한 데이터 분석 및 AI 엔지니어링 역량을 개발하셨습니다.


In [47]:
query = '내가 지원하려고하는 회사 이름은?'
response = rag_chain.invoke({'input': query})

# LLM 답변
print('🧠 GPT-4o-mini 답변:')
print(response['result'])  # MultiRetrievalQAChain는 output_keys=['result']

🧠 GPT-4o-mini 답변:
당신이 지원하려고 하는 회사의 이름은 **임팩티브AI**입니다.


In [52]:
query = '자기소개서 초안을 작성해줘. 문항은 "지원하신 직무에서 성과를 창출하기 위한 본인만의 핵심역량을 제시하고 이것이 (1)직무전문성, (2)조직기여 측면에서 어떻게 발휘될 수 있는지 사례를 들어 설명하시오. (최대 800자 입력가능)'
response = rag_chain.invoke({'input': query})

# LLM 답변
print('🧠 GPT-4o-mini 답변:')
print(response['result'])  # MultiRetrievalQAChain는 output_keys=['result']

🧠 GPT-4o-mini 답변:
저는 데이터 사이언스와 AI 엔지니어링 분야에서의 전문성을 바탕으로 지원한 직무에서 성과를 창출할 수 있는 핵심 역량을 갖추고 있습니다. 특히, 데이터 분석 및 머신러닝 모델 개발에 대한 깊은 이해와 실무 경험이 저의 강점입니다.

첫째, 직무 전문성 측면에서, 저는 Python을 활용한 데이터 전처리 및 분석에 능숙하며, 최근에는 COVID-19 치료 후보물질 발굴 프로젝트에서 사내 플랫폼을 활용하여 유력 후보물질을 성공적으로 발굴했습니다. 이 과정에서 데이터 수집, 정규화, 분석을 주도하며, Scientific Reports 저널에 연구 결과를 제출하는 성과를 이뤘습니다. 이러한 경험은 제가 데이터 기반 의사결정을 지원하고, 효과적인 솔루션을 도출하는 데 기여할 수 있음을 보여줍니다.

둘째, 조직 기여 측면에서, 저는 팀워크와 협업의 중요성을 잘 이해하고 있습니다. 암 조기 진단 모델 개발 프로젝트에서는 cfDNA 및 ctDNA 데이터 수집과 메타데이터 관리 업무를 맡아 3000샘플 이상의 데이터를 수집했습니다. 이를 통해 팀원들과의 원활한 커뮤니케이션을 통해 모델의 성능을 높이는 데 기여했습니다. 이러한 경험은 조직 내에서의 협력과 시너지를 창출하는 데 중요한 역할을 할 것입니다.

결론적으로, 저의 데이터 분석 및 AI 모델 개발 역량은 지원한 직무에서 직접적인 성과를 창출할 수 있는 기반이 될 것이며, 조직의 목표 달성에 기여할 수 있을 것입니다.
